# Demo of the Pillar Assessment Tool

### 0. Setup

First import neccissary modules from `scripts\climate_policy_pipelines\cp1`

In [2]:
# Import necessary modules
import sys
import os
from pathlib import Path

# Get the absolute path of the project root directory
notebook_dir = Path(os.getcwd())  
project_root = notebook_dir.parent.parent  # Go up TWO levels instead of one

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added {project_root} to sys.path")

Added c:\Users\User\GitHub\group-6-final-project to sys.path


In [3]:
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1a_assessment
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1a_assessment_large_context
from scripts.climate_policy_pipelines.cp1.pipeline import run_cp1b_assessment
from scripts.climate_policy_pipelines.cp1.pipeline import get_chunks_for_cp1a

c:\Users\User\GitHub\group-6-final-project\group_rag.venv\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
from scripts.climate_policy_pipelines.cp1.prompts import cp1a_criterion_1_prompt
from scripts.climate_policy_pipelines.cp1.prompts import cp1a_final_assessment_prompt
from scripts.climate_policy_pipelines.cp1.prompts import comprehensive_assessment_prompt

# CP1a Asessment

CP1a is defined as: **Does the country have a framework climate law or equivalent?**

Based on the ASCOR methodology, a country is assessed as ‘Yes’ if it has a framework climate law that fulfils either 1&2&3 or just 4 of 
the following criteria:

1. It sets a strategic direction for decarbonisation

2. It is enshrined in law

3. It sets out at least one of the following obligations

4. Also check this In exceptional cases, the combination of a broad environmental law and a clearly linked executive climate strategy may be sufficient to meet these criteria

Each criteria is handled by a different LLM, which recieves a criteria-specific prompt + retrieved context relevant to that critera.

We can see the prompt for criteria 1 here:

In [5]:
# Print the system message template
print("System Message Template:")
print(cp1a_criterion_1_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(cp1a_criterion_1_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst specializing in climate legislation. 
    Your task is to evaluate whether a climate law sets a strategic direction for decarbonisation.
    
    A law meets this criterion if it includes a clear statement to meet the goals of the Paris Agreement 
    OR a national long-term decarbonisation target.
    
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

     
    Respond with only 'YES' or 'NO' followed by a brief explanation.


Human Message Template:
Context: {context}

Does this law set a strategic direction for decarbonisation?


According to ASCOR methodology, if 1,2,3 are satisfied or 4 is satisfied, then CP1a is answered as Yes.

We therefore use another LLM to evalute the overall prompt based on the other LLM's responses. Lets have a look at its prompt:

In [6]:
# Print the system message template
print("System Message Template:")
print(cp1a_final_assessment_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(cp1a_final_assessment_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst making a final assessment of climate legislation.
    
    A country is assessed as 'YES' for having framework climate law if:
    - Criteria 1, 2, AND 3 are all satisfied, OR
    - Criterion 4 is satisfied (exceptional case)
   
    For any claims you make, you **MUST** include the page number and document citation in the format (page X, doc Y).

    Based on the individual assessments, provide a final 'YES' or 'NO' answer with reasoning.


Human Message Template:
Individual criterion assessments:
    Criterion 1 (Strategic direction): {criterion_1_result}
    Criterion 2 (Enshrined in law): {criterion_2_result}
    Criterion 3 (Obligations): {criterion_3_result}
    Criterion 4 (Exceptional case): {criterion_4_result}
    
    What is the final assessment?



### What content does the assessment use?

Lets have a look at the chunks it retrieves in order to inform its assessment of each criteria.

If you look at `get_chunks_for_cp1a` in `scripts/climate_policy_pipelines/cp1.pipeline`, you can see that these are 4 sentences it embeds and retrives chunks for, retrieving 100 chunks for each CP1a criteria 

```
cp1a_prompts = [
    "strategic direction for decarbonisation Paris Agreement national long-term target",
    "climate law enshrined in law legally binding framework",
    "obligations carbon budgets emissions targets monitoring requirements",
    "environmental law executive climate strategy broad framework"
]
```

Lets have a look at just what `cp1a_criteria_1_search_prompt` retrieves:

In [ ]:
cp1a_criteria_1_search_prompt = "strategic direction for decarbonisation Paris Agreement national long-term target"
get_chunks_for_cp1a(cp1a_criteria_1_search_prompt)

### 3. Demo assessment 

Lets run an assessment for Albania

In [7]:
run_cp1a_assessment(country="ALB")

Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

AIMessage(content='Based on the individual criterion assessments, the final assessment is: **YES**\n\nThe country satisfies Criteria 1, 2, and 3, which are all required for a \'YES\' assessment. Specifically:\n\n* Criterion 1 (Strategic direction) is satisfied, as the law includes a statement to fulfill reporting requirements under the UNFCCC and the Paris Agreement, and mentions a national long-term decarbonisation target (page not specified, Albania’s National Adaptation Plan First - progress report and National Energy and Climate Plan 2019 Draft).\n* Criterion 2 (Enshrined in law) is satisfied, as climate legislation is enshrined in law through the "Law for Climate Change" (page not specified, National Energy and Climate Plan 2019 Draft).\n* Criterion 3 (Obligations) is satisfied, as the law sets out specific obligations, such as developing policy instruments and public spending in support of climate goals (page not specified, National Energy and Climate Plan 2019 Draft).\n\nAdditio

# CP1a Large Context Window Asessment

Another tool just uses one, more powerful large context window LLM to assess all components of CP1a togehter instead of seperatley. Here is the prompt:

In [8]:
# Print the system message template
print("System Message Template:")
print(comprehensive_assessment_prompt.messages[0].prompt.template)

print("\n" + "="*50 + "\n")

# Print the human message template  
print("Human Message Template:")
print(comprehensive_assessment_prompt.messages[1].prompt.template)

System Message Template:
You are an expert legal analyst specializing in climate legislation assessment. 

Your task is to evaluate whether a country has a framework climate law based on specific criteria and provide a structured markdown assessment.

EVALUATION CRITERIA:
A country is assessed as 'YES' if it has a framework climate law that fulfils ALL of criteria 1, 2, AND 3, OR criterion 4:

1. STRATEGIC DIRECTION: Sets a strategic direction for decarbonisation (must include a clear statement to meet the goals of the Paris Agreement OR a national long-term decarbonisation target)

2. ENSHRINED IN LAW: Is enshrined in law (must be legislative rather than executive, except in particular political systems)

3. OBLIGATIONS: Sets out at least one of the following obligations:
   - Meeting a national target
   - Developing, revising, implementing or complying with domestic plans, strategies or policies
   - Developing policy instruments such as regulation, taxation or public spending in su

Lets see how it performs on the same country:

In [10]:
run_cp1a_assessment_large_context(country="ALB")


Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

Large Context Assessment:
# Climate Legislation Assessment: CP 1.a Framework Climate Law

## Individual Criterion Evaluation

### Criterion 1: Strategic Direction for Decarbonisation
**Result:** YES
**Reasoning:** The National Energy and Climate Plan 2019 Draft mentions that "Setting up the ETS in Albania will contribute to reach the NDC target for reduction of GHG emission from the industry sector as well as the NECP target for decarbonisation of the economy" (page not specified, doc National Energy and Climate Plan 2019 Draft). Additionally, the document mentions that the country aims to "fulfill the reporting requirements under the United Nations Framework Convention on Climate Change and the Paris Agreement" (page not specified, doc Albania’s National Adaptation Plan First - progress report). This suggests that the country has a strategic direction for decarbonisation aligned with the Paris Agreement.

### Criterion 2: Enshrined in Law
**Result:** YES
**Reasoning:** The National En

AIMessage(content='# Climate Legislation Assessment: CP 1.a Framework Climate Law\n\n## Individual Criterion Evaluation\n\n### Criterion 1: Strategic Direction for Decarbonisation\n**Result:** YES\n**Reasoning:** The National Energy and Climate Plan 2019 Draft mentions that "Setting up the ETS in Albania will contribute to reach the NDC target for reduction of GHG emission from the industry sector as well as the NECP target for decarbonisation of the economy" (page not specified, doc National Energy and Climate Plan 2019 Draft). Additionally, the document mentions that the country aims to "fulfill the reporting requirements under the United Nations Framework Convention on Climate Change and the Paris Agreement" (page not specified, doc Albania’s National Adaptation Plan First - progress report). This suggests that the country has a strategic direction for decarbonisation aligned with the Paris Agreement.\n\n### Criterion 2: Enshrined in Law\n**Result:** YES\n**Reasoning:** The National

# CP1b Assessment

We have also built a tool that automatically evalutes CP1b

CP1b is defined by ASCOR as: **Does the country’s framework climate law specify key accountability elements?**


A country is assessed as ‘Yes’ if its framework climate law contains all three of the following accountability elements: 
1. Specification of who is accountable to whom for at least one stated obligation (e.g. accountability of executive to parliament, or private parties to executive authorities) 
2. Specification of how compliance is assessed for at least one stated obligation (e.g. transparency mechanisms in the form of monitoring, reporting and verification, parliamentary oversight, expert assessments, court proceedings) 
3. Specification of what happens in the case of non-compliance for at least one stated obligation (e.g. parliamentary intervention, judicial orders, financial penalties). 

Like with CP1a, these are evaluated separately and then evaluted together by a evaluator LLM. There is additional guidance in the methodology on how to asses these criteria wich is included in the prompts (see `scripts/climate_policy_pipeline/cp1/prompts`)

Lets see how the tool performs:

In [11]:
run_cp1b_assessment(country="ALB")

Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at /Users/jessiefung/Desktop/DS205/group-6-final-project/local_model/climatebert/distilroberta-base-climate-f and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You 

Detailed Assessment:
Based on the individual criterion assessments, the final assessment is:

**NO**

Reasoning: Although the country's framework climate law meets Criteria 1 and 2, it does not meet Criterion 3, which requires specification of what happens in the case of non-compliance for at least one stated obligation. Since the documents provided do not mention any consequences or actions to be taken in case of non-compliance, the country's framework climate law does not fully meet the accountability requirements.


AIMessage(content="Based on the individual criterion assessments, the final assessment is:\n\n**NO**\n\nReasoning: Although the country's framework climate law meets Criteria 1 and 2, it does not meet Criterion 3, which requires specification of what happens in the case of non-compliance for at least one stated obligation. Since the documents provided do not mention any consequences or actions to be taken in case of non-compliance, the country's framework climate law does not fully meet the accountability requirements.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 555, 'total_tokens': 652, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'system_fingerprint': None, 'id': 'chatcmpl-fac61a06d7574f8284957e8dc041ae8a', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--6b31214b-f45e-4d45-b013-4a82b9fee599-0', usage_metadata=

If you want to easily automate assessment so do not want to have justification and only output the yes/no answer, you can set `detailed=False` in the function:

In [ ]:
run_cp1b_assessment(country='ALB', detailed=False)

Note that the assessment will be the same always because model temperature is set to 0.